In [ ]:
# %pip install beautifulsoup4
# %pip install requests

In [ ]:
# %pip install urllib3==1.26.7

In [1]:
import csv
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse

In [2]:
# def generate_page_url(base_url, page_number):
#     page_url = base_url.replace('/page=1/', f'/page={page_number}/')
#     return page_url

def generate_page_url(base_url, page_number):
    url_parts = base_url.split('=')
    url_parts[-1] = str(page_number)
    page_url = '='.join(url_parts)
    return page_url


In [3]:
# Scrape Link to Detail Page
links_array = []
dashboard_url = 'https://www.lamudi.co.id/west-nusa-tenggara/house/buy/?page=1'
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 3)

for page_number in range(1, 21):  # Ubah 20 menjadi 21 untuk memperoleh halaman 1 hingga 20
    current_page_url = generate_page_url(dashboard_url, page_number)
    driver.get(current_page_url)

    # STATIS
    # soup = BeautifulSoup(driver.page_source, 'html.parser')  # Menggunakan driver.page_source untuk konten halaman
    # links = soup.find_all('a', class_='ListingCell-moreInfo-button-v2_redesign')
    # for link in links:
    #     print(link['href'])

    # Tunggu hingga konten dinamis selesai dimuat
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'ListingCell-moreInfo-button-v2_redesign')))
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    links = soup.find_all('a', class_='ListingCell-moreInfo-button-v2_redesign')
    # for link in links:
    #     print(link['href'])
    # print(len(links))

    # DENGAN ARRAY
    for link in links:
        links_array.append(link['href'])  # Tambahkan link ke dalam array
        
    # print(len(links))

csv_file = 'data.csv'

def scrape_detail(url):
    bedrooms = bathrooms = building_size = land_size = '0'  # Default value

    try:
        bedrooms_element = soup.find('div', {'data-attr-name': 'bedrooms'}).find_next('div')
        bedrooms = bedrooms_element.text.strip() if bedrooms_element else '0'
    except AttributeError:
        pass

    try:
        bathrooms_element = soup.find('div', {'data-attr-name': 'bathrooms'}).find_next('div')
        bathrooms = bathrooms_element.text.strip() if bathrooms_element else '0'
    except AttributeError:
        pass

    try:
        building_size_element = soup.find('div', {'data-attr-name': 'building_size'}).find_next('div')
        building_size = building_size_element.text.strip() if building_size_element else '0'
    except AttributeError:
        pass

    try:
        land_size_element = soup.find('div', {'data-attr-name': 'land_size'}).find_next('div')
        land_size = land_size_element.text.strip() if land_size_element else '0'
    except AttributeError:
        pass

    try:
        price = soup.find('div', {'Title-pdp-price'}).find('span', class_='FirstPrice')
        prices = price.text.strip() if price else '0'
    except AttributeError:
        pass

    return bedrooms, bathrooms, building_size, land_size, prices

# Scrape dan simpan dalam CSV
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Jumlah Kamar Tidur', 'Jumlah Kamar Mandi', 'Luas Bangunan', 'Luas Tanah','Harga'])

    for link in links_array[:600]:  # Mengambil 600 data pertama
        driver.get(link)  # Buka halaman detail rumah berdasarkan link
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        bedrooms, bathrooms, building_size, land_size, prices = scrape_detail(soup)
        writer.writerow([bedrooms, bathrooms, building_size, land_size, prices])

print('Data telah disimpan dalam file CSV.')

driver.quit()  # Tutup browser setelah selesai scraping


Data telah disimpan dalam file CSV.
